In [2]:
%pip install pyspark
%pip install findspark
import findspark
import pyspark
findspark.init()
from pyspark.sql import SparkSession


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845499 sha256=131f2c082587039fde80d64d0197d8fa6620cdb9d7510fa29bd21b6544f056d6
  Stored in directory: /home/vscode/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

In [7]:
data_df = spark.read.csv('content/data.csv', header=True, inferSchema=True)
data_df

UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
4,AF,AFG,4,null,null,null,Afghanistan,33.93911,67.709953,Afghanistan,38928341
8,AL,ALB,8,null,null,null,Albania,41.1533,20.1683,Albania,2877800
10,AQ,ATA,10,null,null,null,Antarctica,-71.9499,23.347,Antarctica,null
12,DZ,DZA,12,null,null,null,Algeria,28.0339,1.6596,Algeria,43851043
20,AD,AND,20,null,null,null,Andorra,42.5063,1.5218,Andorra,77265
24,AO,AGO,24,null,null,null,Angola,-11.2027,17.8739,Angola,32866268
28,AG,ATG,28,null,null,null,Antigua and Barbuda,17.0608,-61.7964,Antigua and Barbuda,97928
32,AR,ARG,32,null,null,null,Argentina,-38.4161,-63.6167,Argentina,45195777
51,AM,ARM,51,null,null,null,Armenia,40.0691,45.0382,Armenia,2963234
40,AT,AUT,40,null,null,null,Austria,47.5162,14.5501,Austria,9006400
